In [ ]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pylab import mpl
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rc
rc('font', family='DejaVu Sans')
rc('axes', unicode_minus=True)  # 启用负号支持
from matplotlib.ticker import FormatStrFormatter

In [ ]:
# 载入数据集（这里以Iris数据集为例）
data_train = pd.read_excel(io=r'P-X.xlsx',sheet_name=2)
data_train=data_train.iloc[:,4:]
features_train = data_train.iloc[:,:-1]
target_train = data_train.iloc[:,-1]
X = features_train
y = target_train

# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 归一化：使用StandardScaler将数据标准化（均值为0，标准差为1）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
fcols = 6
frows = len(features_train.columns)
plt.figure(figsize=(6*fcols,4*frows),dpi=450)

i=0
for col in features_train.columns:
    i+=1
    ax=plt.subplot(frows,fcols,i)
    sns.distplot(features_train[col],color='#EF82A0',hist_kws={"rwidth":0.9})
    plt.xlabel(col)

In [ ]:
import xgboost
from xgboost import XGBRFRegressor
xgbr = XGBRFRegressor()
xgbr.fit(features_scal,target_scal)
importance = xgbr.feature_importances_
col = zip(features_scal,importance)
importance_data = pd.DataFrame(col,columns=['features','importance'])
importance_data_asc = importance_data.sort_values("importance",ascending=True)
fig = plt.figure(figsize=(20,15),dpi=500)
plt.barh(y=importance_data_asc['features'],width=importance_data_asc['importance'],height=0.9)
plt.xlabel("importance",fontsize=15)
plt.ylabel("features",fontsize=15)
plt.show()

In [ ]:
# 初始化 RandomForestRegressor 模型
model = RandomForestRegressor(n_estimators=100, random_state=42)

# 使用RFECV进行递归特征消除，自动进行交叉验证
rfecv = RFECV(estimator=model, step=1, cv=10, scoring='neg_root_mean_squared_error')
rfecv.fit(X_train_scaled, y_train)

# 获取每个特征数量对应的负RMSE（注意：RFECV返回的是负值）
rmses = -rfecv.grid_scores_

# 通过np.unravel_index找到最小RMSE对应的索引
ind = np.unravel_index(np.argmin(rmses, axis=None), rmses.shape)
optimal_feature_count = ind[0] + 1  # +1 因为特征数量从1开始
optimal_rmse = rmses[ind]

# 获取排名前optimal_feature_count的特征的列名
ranking = rfecv.ranking_  # 获取每个特征的排名
selected_features_indices = np.argsort(ranking)[:optimal_feature_count]  # 排名最前的特征

# 获取最优特征集的列名
optimal_features = X.columns[selected_features_indices]

# 输出最优特征集的列名
print(f"Optimal number of features: {optimal_feature_count}")
print(f"Optimal RMSE: {optimal_rmse}")
print(f"Optimal features selected: {optimal_features.tolist()}")

# 训练最优特征集的模型
model.fit(X_train_scaled[:, selected_features_indices], y_train)

# 用测试集数据进行预测
y_pred_scaled = model.predict(X_test_scaled[:, selected_features_indices])

# 反归一化预测值
y_pred = scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))

# 反归一化真实目标值
y_test_inv = scaler.inverse_transform(y_test.values.reshape(-1, 1))

# 计算反归一化后的RMSE
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred))
print(f"RMSE on original scale: {rmse}")

# 绘制交叉验证中每个特征数量下的RMSE曲线
plt.figure(figsize=(10, 7), dpi=300)  # 设置图表大小和分辨率
plt.plot(range(1, len(rmses) + 1), rmses, marker='o', linestyle='-', color='darkblue', linewidth=2, markersize=8)

# 标记出最优点，使用红色五角星
plt.scatter(optimal_feature_count, optimal_rmse, color='red', marker='*', s=200, zorder=5, label='Optimal Feature Count')

# 设置标题和坐标轴标签
plt.title('RFECV: Number of Features vs RMSE (Cross-validation)', fontsize=16)
plt.xlabel('Number of Features Selected', fontsize=14)
plt.ylabel('Root Mean Squared Error (RMSE)', fontsize=14)

# 设置坐标轴刻度
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# 优化网格线，背景为灰色
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')

# 图例
plt.legend(loc='upper right', fontsize=12)

# 紧凑布局
plt.tight_layout()

# 显示图形
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# 创建图
G = nx.Graph()

# 定义层节点
input_layer = [f'I{i+1}' for i in range(5)]  # 输入层前5个节点
hidden_layer1 = ['H1', 'H2', 'H3', 'H4', 'H5']  # 隐藏层1
hidden_layer2 = ['H6', 'H7', 'H8', 'H9', 'H10']  # 隐藏层2
output_layer = ['O1']  # 输出层1个节点

# 用省略号表示中间层的节点
input_layer.append('...')
hidden_layer1.append('...')
hidden_layer2.append('...')
output_layer.append('...')

# 添加节点到图中
G.add_nodes_from(input_layer + hidden_layer1 + hidden_layer2 + output_layer)

# 添加边（连接层之间的节点）
for i in input_layer:
    if i != '...':  # 忽略省略号节点
        for h in hidden_layer1:
            if h != '...':
                G.add_edge(i, h, weight=f'W_{i[1]}{h[1]}')
for h1 in hidden_layer1:
    if h1 != '...':
        for h2 in hidden_layer2:
            if h2 != '...':
                G.add_edge(h1, h2, weight=f'W_{h1[1]}{h2[1]}')
for h2 in hidden_layer2:
    if h2 != '...':
        for o in output_layer:
            if o != '...':
                G.add_edge(h2, o, weight=f'W_{h2[1]}{o[1]}')

# 设置图形布局
pos = {}
pos.update((node, (1, i)) for i, node in enumerate(input_layer))  # 输入层
pos.update((node, (2, i)) for i, node in enumerate(hidden_layer1))  # 隐藏层1
pos.update((node, (3, i)) for i, node in enumerate(hidden_layer2))  # 隐藏层2
pos.update((node, (4, 2)) for node in output_layer)  # 输出层调整至中间位置

# 绘制图
plt.figure(figsize=(12, 8), dpi=1200)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color='skyblue', font_size=12, font_weight='bold', edge_color='gray', width=1.5)

# 美化：修改文字位置和风格
plt.text(1, 4.3, "Input Layer", fontsize=16, ha='center', fontweight='bold', color='black')
plt.text(2, 4.3, "Hidden Layer", fontsize=16, ha='center', fontweight='bold', color='black')
plt.text(3, 4.3, "Hidden Layer", fontsize=16, ha='center', fontweight='bold', color='black')
plt.text(4, 4.3, "Output Layer", fontsize=16, ha='center', fontweight='bold', color='black')

# 添加权重标注
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10, font_color='darkred')

# 使图形更简洁并优化显示
plt.title('Neural Network Architecture', fontsize=20, fontweight='bold', color='black')
plt.axis('off')  # 关闭坐标轴

# 显示
plt.show()


In [ ]:
# 创建一个包含训练集和测试集真实值与预测值的数据框
data_train = pd.DataFrame({    
    'True': y_train,    
    'Predicted': y_pred_train,    
    'Data Set': 'Train'
})
data_test = pd.DataFrame({    
    'True': y_test,    
    'Predicted': y_pred_test,    
    'Data Set': 'Test'
})
data = pd.concat([data_train, data_test])
# 自定义调色板
palette = {'Train': '#b4d4e1', 'Test': '#f4ba8a'}
# 创建 JointGrid 对象
plt.figure(figsize=(8, 6), dpi=1200)
g = sns.JointGrid(data=data, x="True", y="Predicted", hue="Data Set", height=10, palette=palette)
# 绘制中心的散点图
g.plot_joint(sns.scatterplot, alpha=0.5)
# 添加训练集的回归线
sns.regplot(data=data_train, x="True", y="Predicted", scatter=False, ax=g.ax_joint, color='#b4d4e1', label='Train Regression Line')
# 添加测试集的回归线
sns.regplot(data=data_test, x="True", y="Predicted", scatter=False, ax=g.ax_joint, color='#f4ba8a', label='Test Regression Line')
# 添加边缘的柱状图
g.plot_marginals(sns.histplot, kde=False, element='bars', multiple='stack', alpha=0.5)
# 关闭 y 轴的边缘柱状图
g.ax_marg_y.set_visible(False)
# 添加拟合优度文本在右下角
ax = g.ax_joint
ax.text(0.95, 0.1, f'Train $R^2$ = {r2_train:.3f}', transform=ax.transAxes, fontsize=12,        
        verticalalignment='bottom', horizontalalignment='right', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))
ax.text(0.95, 0.05, f'Test $R^2$ = {r2_test:.3f}', transform=ax.transAxes, fontsize=12,        
        verticalalignment='bottom', horizontalalignment='right', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))
# 在左上角添加模型名称文本
ax.text(0.75, 0.99, 'Model = xgboost', transform=ax.transAxes, fontsize=12,        
        verticalalignment='top', horizontalalignment='left', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))
# 添加中心线
ax.plot([data['True'].min(), data['True'].max()], [data['True'].min(), data['True'].max()], c="black", alpha=0.5, linestyle='--', label='x=y')
ax.legend()
plt.show()

In [ ]:
all_target_value = X
fig = plt.figure(dpi=600, figsize=(6, 6))
sns.set_palette("hls")
sns.set(context='paper', style='ticks', font_scale=2)
sns.displot(all_target_value["C"], kde=True, alpha=1, color='r', height=5, aspect=1.5)
plt.savefig(bar_plot_savepath + 'C.png')
plt.close()

In [ ]:
# 定义一个字典来存储每个模型的结果
results = {}

# 模型列表
models = {
    'Elastic Net': ElasticNet(),
    'KNN': KNeighborsRegressor(),
    'DT': DecisionTreeRegressor(),
    'RF': RandomForestRegressor(),
    'GB': GradientBoostingRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'LightGBM': lgb.LGBMRegressor(),
    'SVM': SVR(),
    'NN': MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500)  # 使用MLPRegressor
}

# 对每个模型进行训练和评估
for model_name, model in models.items():
    model.fit(X_train_scaled, y_train_scaled.ravel())  # 对所有模型统一训练
    y_pred_scaled = model.predict(X_test_scaled)  # 预测测试集数据

    # 反归一化
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
    y_test_inv = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1))

    # 计算RMSE和MAE
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred))
    mae = mean_absolute_error(y_test_inv, y_pred)

    # 保存结果
    results[model_name] = {
        'RMSE': rmse,
        'MAE': mae
    }

# 将结果转化为DataFrame
results_df = pd.DataFrame(results).T

# 绘制柱状图
fig, ax = plt.subplots(figsize=(10, 6))

# 绘制RMSE
results_df['RMSE'].plot(kind='bar', width=0.4, position=1, ax=ax, color='b', label='RMSE')

# 绘制MAE
results_df['MAE'].plot(kind='bar', width=0.4, position=0, ax=ax, color='r', label='MAE')

ax.set_ylabel('Error Value')
ax.set_title('Model Comparison on RMSE and MAE')
ax.set_xticklabels(results_df.index, rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.show()

# 打印结果表
print(results_df)


In [ ]:
# 1. 从Excel文件中读取数据
file_path = 'data.xlsx'  # 请将文件路径改为你实际的路径
df = pd.read_excel(file_path)

# 2. 处理数据（假设数据的最后一列是标签，去掉最后一列）
X = df.iloc[:, :-1].values  # 提取特征数据（去掉最后一列标签列）

# 3. 初始化KMeans并使用K-Means++聚类算法
k = 3  # 分成3类
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)

# 4. 训练KMeans模型
kmeans.fit(X)

# 5. 获取聚类结果
labels = kmeans.labels_  # 每个数据点的聚类标签
centers = kmeans.cluster_centers_  # 聚类中心

# 6. 使用t-SNE进行降维（降到3维）
tsne = TSNE(n_components=3, random_state=42)
X_tsne = tsne.fit_transform(X)

# 7. 绘制三维散点图
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 绘制散点图，颜色根据聚类标签
scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=labels, cmap='viridis', s=60, alpha=0.7, edgecolors='w', linewidth=0.5)

# 标注聚类中心
ax.scatter(centers[:, 0], centers[:, 1], centers[:, 2], c='red', marker='X', s=200, label='Centroids')

# 添加标题和标签
ax.set_title("K-Means++ Clustering Results (t-SNE)", fontsize=16)
ax.set_xlabel('t-SNE Component 1', fontsize=12)
ax.set_ylabel('t-SNE Component 2', fontsize=12)
ax.set_zlabel('t-SNE Component 3', fontsize=12)

# 显示颜色条
fig.colorbar(scatter, label='Cluster Label')

# 显示图例
ax.legend(loc='upper right', fontsize=12)

# 显示图形
plt.show()

# 8. 输出聚类中心
print("聚类中心：")
print(centers)


In [ ]:
# 1. 从Excel文件中读取数据
file_path = 'data.xlsx'  # 请将文件路径改为你实际的路径
df = pd.read_excel(file_path)

# 2. 处理数据（假设数据的最后一列是标签，去掉最后一列）
X = df.iloc[:, :-1].values  # 提取特征数据（去掉最后一列标签列）

# 3. 初始化KMeans并使用K-Means++聚类算法
k = 3  # 分成3类
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)

# 4. 训练KMeans模型
kmeans.fit(X)

# 5. 获取聚类结果
labels = kmeans.labels_  # 每个数据点的聚类标签
centers = kmeans.cluster_centers_  # 聚类中心

# 6. 使用t-SNE进行降维（降到3维）
tsne = TSNE(n_components=3, random_state=42)
X_tsne = tsne.fit_transform(X)

# 7. 绘制三维散点图
fig = plt.figure(figsize=(12, 10), dpi=150)  # 提升图像清晰度，设置更高的DPI
ax = fig.add_subplot(111, projection='3d')

# 绘制散点图，颜色根据聚类标签
scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=labels, cmap='viridis', s=50, alpha=0.8, edgecolors='w', linewidth=0.5)

# 标注聚类中心
ax.scatter(centers[:, 0], centers[:, 1], centers[:, 2], c='red', marker='X', s=200, label='Centroids')

# 设置3D视角，使图形朝向右上
ax.view_init(elev=20, azim=45)

# 添加标题和标签
ax.set_title("K-Means++ Clustering Results (t-SNE)", fontsize=18)
ax.set_xlabel('t-SNE Component 1', fontsize=14)
ax.set_ylabel('t-SNE Component 2', fontsize=14)
ax.set_zlabel('t-SNE Component 3', fontsize=14)

# 显示颜色条
fig.colorbar(scatter, label='Cluster Label')

# 显示图例
ax.legend(loc='upper right', fontsize=12)

# 显示图形
plt.show()

# 8. 输出聚类中心
print("聚类中心：")
print(centers)


In [ ]:
corr = data.corr(method='pearson')
n = len(data.columns)
fig, axes = plt.subplots(n, n, figsize=(2.5 * n, 2.5 * n))
# 绘制每个位置的散点图和直方图
for i in range(n):    
    for j in range(n):        
        ax = axes[i, j]        
        if i == j:            
            # 对角线：绘制直方图            
            sns.histplot(data.iloc[:, i], kde=True, ax=ax)        
        elif i > j:            
            # 下三角：绘制散点图            
            sns.scatterplot(x=data.iloc[:, j], y=data.iloc[:, i], ax=ax)        
        else:            
        # 上三角：绘制热图显示相关系数            
            sns.heatmap(pd.DataFrame([[corr.iloc[i, j]]]), cmap=sns.diverging_palette(240, 10, as_cmap=True),                        
                        cbar=False, annot=True, fmt=".2f", square=True, ax=ax, vmin=-1, vmax=1,                        
                        annot_kws={"size": 20})  # 设置相关系数数字字体大小
        # 隐藏不需要的轴标签        
        if i < n - 1:            
            ax.xaxis.set_visible(False)      
        if j > 0:            
            ax.yaxis.set_visible(False)
# 调整子图之间的间距
plt.subplots_adjust(hspace=0.2, wspace=0.2)
# 在图形旁边添加全局色条
fig.subplots_adjust(right=0.85)  # 调整图形右侧空间以显示色条
cbar_ax = fig.add_axes([0.87, 0.15, 0.03, 0.7])  # 定义色条位置和大小
norm = plt.Normalize(vmin=-1, vmax=1)
sm = plt.cm.ScalarMappable(cmap=sns.diverging_palette(240, 10, as_cmap=True), norm=norm)
sm.set_array([])  # 为空数组设置色条
fig.colorbar(sm, cax=cbar_ax)  # 添加全局色条
plt.savefig("改进后的可视化.pdf", format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
# 创建相关性矩阵
corr = data.corr()
# 创建图形
fig, ax = plt.subplots(figsize=(20, 20), dpi=600)
# 使用颜色映射：使用 'coolwarm' colormap
cmap = plt.cm.viridis
norm = plt.Normalize(vmin=-1, vmax=1)
# 初始化一个空的可绘制对象用于颜色条
scatter_handles = []
# 循环绘制气泡图和数值
for i in range(len(corr.columns)):    
    for j in range(len(corr.columns)):        
        if i > j:  # 对角线左下部分，只显示气泡            
            color = cmap(norm(corr.iloc[i, j]))  # 根据相关系数获取颜色            
            scatter = ax.scatter(i, j, s=np.abs(corr.iloc[i, j]) * 1000, color=color, alpha=0.75)            
            scatter_handles.append(scatter)  # 保存scatter对象用于颜色条        
        elif i < j:  # 对角线右上部分，只显示数值            
            color = cmap(norm(corr.iloc[i, j]))  # 数值的颜色同样基于相关系数            
            ax.text(i, j, f'{corr.iloc[i, j]:.2f}', ha='center', va='center', color=color, fontsize=10)        
        else:  # 对角线部分，显示空白            
            ax.scatter(i, j, s=1, color='white')
# 设置坐标轴标签
ax.set_xticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=45, ha='right', fontsize=10)
ax.set_yticks(range(len(corr.columns)))
ax.set_yticklabels(corr.columns, fontsize=10)
# 添加颜色条
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # 仅用于显示颜色条
fig.colorbar(sm, ax=ax, label='Correlation Coefficient')
# 添加标题和布局调整
plt.tight_layout()
plt.savefig("Bubble Heatmap with Color Mapping for Bubbles and Numbers.pdf", format='pdf', bbox_inches='tight')
plt.show()

In [ ]:

xgbr = XGBRegressor()
xgbr_grid = {
    'min_child_weight':[1,5,10],
    'n_estimators':[30,40,50],
    'max_depth':[4,5,6,8],
    'learning_rate':[0.1,0.2,0.3,0.4,0.001,0.005,0.5,1]}
fold = RepeatedKFold(n_repeats=1,n_splits=10,random_state=30)
xgbr_grid_cv = GridSearchCV(estimator=xgbr,cv=fold,param_grid=xgbr_grid,n_jobs=-1)
xgbr_grid_cv.fit(features,target)
xgbr_optimized = xgbr_grid_cv.best_params_#得到最优参数
xgbr_optimized_estimator = xgbr_grid_cv.best_estimator_#最优参数实例化
model = xgbr_optimized_estimator.fit(x_train,y_train)

In [ ]:
fig = plt.figure(figsize=(5,5),dpi=600)
plt.scatter(y_train,xgbr_preds_train,alpha=1,marker='o',s=35,color='red',facecolor="None",label='Train_set')
plt.scatter(y_test,xgbr_preds_1,alpha=1,marker='h',s=35,color='g',facecolor="None",label='Test_set')
plt.plot([-0.2,0.4],[-0.2,0.4],c='black',ls='--')
plt.xlabel('DFT',fontsize=20)
plt.ylabel('ML',fontsize=20)
plt.legend()
plt.show()

In [ ]:

def main(random_ststes,r2_train_list,r2_test_list):
    xgbr = XGBRegressor()
    xgbr_grid = {
        'min_child_weight':[1],
        'n_estimators':[50],
        'max_depth':[8],
        'learning_rate':[0.31]}
    fold = RepeatedKFold(n_repeats=1,n_splits=10,random_state=30)
    xgbr_grid_cv = GridSearchCV(estimator=xgbr,cv=fold,param_grid=xgbr_grid,n_jobs=-1)
    #xgbr_optimized_estimator = xgbr_grid_cv.best_estimator_
    x_train,x_test,y_train,y_test = train_test_split(features,target,train_size=0.9,random_state=random_ststes)
    model = xgbr_grid_cv.fit(x_train,y_train)

    xgbr_preds_1 = model.predict(x_test)
    xgbr_preds_train = model.predict(x_train)
    
    r2_train = r2_score(y_train,xgbr_preds_train)
    r2_test = r2_score(y_test,xgbr_preds_1)
    r2_train_list.append(r2_train)
    r2_test_list.append(r2_test)

In [ ]:
r2_train_listss= []
r2_test_listss= []
for i in range(200):
    main(i,r2_train_listss,r2_test_listss)
data_1 = pd.DataFrame({"train_R2":r2_train_listss,"test_R2":r2_test_listss})
data_1.to_csv("random_xgboost_200.csv")

In [ ]:
# MLPRegressor GridSearchCV 调参
nn_model = MLPRegressor(max_iter=500, random_state=42)

nn_param_grid = {
    'hidden_layer_sizes': [(64, 32), (128, 64), (256, 128)],  # 隐藏层的神经元数量
    'activation': ['relu', 'tanh'],  # 激活函数
    'solver': ['adam', 'lbfgs'],               # 求解器
    'learning_rate_init': [0.001, 0.01]  # 初始学习率
}

# 使用 GridSearchCV 进行网格搜索
nn_grid_search = GridSearchCV(estimator=nn_model, param_grid=nn_param_grid, cv=10, n_jobs=-1, verbose=1)
nn_grid_search.fit(X_train_scaled, y_train_scaled.values.ravel())

# 输出最佳参数
print(f"NN Best Parameters: {nn_grid_search.best_params_}")
nn_best_model = nn_grid_search.best_estimator_

# 使用最佳模型对训练集进行预测
y_train_pred_scaled_nn = nn_best_model.predict(X_train_scaled)
y_train_pred_nn = m2.inverse_transform(y_train_pred_scaled_nn.reshape(-1, 1))

# 使用最佳模型对测试集进行预测
y_test_pred_scaled_nn = nn_best_model.predict(X_test_scaled)
y_test_pred_nn = m4.inverse_transform(y_test_pred_scaled_nn.reshape(-1, 1))


In [ ]:
# Random Forest GridSearchCV 调参
rf_model = RandomForestRegressor(random_state=42)

rf_param_grid = {
    'n_estimators': [100, 200],       # 树的数量
    'max_depth': [5, 10, None],        # 最大深度
    'min_samples_split': [2, 5],       # 分裂节点的最小样本数
    'min_samples_leaf': [1, 2]         # 叶子节点的最小样本数
}

# 使用 GridSearchCV 进行网格搜索
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=10, n_jobs=-1, verbose=1)
rf_grid_search.fit(X_train_scaled, y_train_scaled.values.ravel())

# 输出最佳参数
print(f"RF Best Parameters: {rf_grid_search.best_params_}")
rf_best_model = rf_grid_search.best_estimator_

# 使用最佳模型对训练集进行预测
y_train_pred_scaled_rf = rf_best_model.predict(X_train_scaled)
y_train_pred_rf = m2.inverse_transform(y_train_pred_scaled_rf.reshape(-1, 1))

# 使用最佳模型对测试集进行预测
y_test_pred_scaled_rf = rf_best_model.predict(X_test_scaled)
y_test_pred_rf = m4.inverse_transform(y_test_pred_scaled_rf.reshape(-1, 1))

In [ ]:
# XGBoost
xgboost_model = XGBRegressor(random_state=42)

xgboost_param_grid = {
    'n_estimators': [100, 200],  # 树的数量
    'max_depth': [3, 4, 5],       # 树的最大深度
    'learning_rate': [0.01, 0.1], # 学习率
    'subsample': [0.8, 1.0]       # 样本子集比例
}

# 使用 GridSearchCV 进行网格搜索
xgboost_grid_search = GridSearchCV(estimator=xgboost_model, param_grid=xgboost_param_grid, cv=10, n_jobs=-1, verbose=1)
xgboost_grid_search.fit(X_train_scaled, y_train_scaled.values.ravel())

# 输出最佳参数
print(f"XGBoost Best Parameters: {xgboost_grid_search.best_params_}")
xgboost_best_model = xgboost_grid_search.best_estimator_

# 使用最佳模型对训练集进行预测
y_train_pred_scaled = xgboost_best_model.predict(X_train_scaled)
y_train_pred = m2.inverse_transform(y_train_pred_scaled.reshape(-1, 1))

# 使用最佳模型对测试集进行预测
y_test_pred_scaled = xgboost_best_model.predict(X_test_scaled)
y_test_pred = m4.inverse_transform(y_test_pred_scaled.reshape(-1, 1))



In [ ]:
# HR计算函数
def calculate_hit_rate(actual, predicted, threshold=0.002):
    """计算命中率HR，预测值与实际值的差值不超过0.0015视为命中"""
    hit = abs(actual - predicted) <= threshold
    return hit.mean()

# 计算RMSE和MAE
def calculate_rmse_mae(actual, predicted):
    """计算RMSE和MAE"""
    rmse = mean_squared_error(actual, predicted, squared=False)
    mae = mean_absolute_error(actual, predicted)
    return rmse, mae

# 创建一个绘制函数
def plot_joint_grid_from_excel(train_file, test_file, model_name):
    # 读取训练集和测试集结果
    train_df = pd.read_excel(train_file)
    test_df = pd.read_excel(test_file)

    # 计算命中率HR
    hr_train = calculate_hit_rate(train_df['Actual_Train'], train_df.iloc[:, 1])
    hr_test = calculate_hit_rate(test_df['Actual_Test'], test_df.iloc[:, 1])
    
    # 计算RMSE和MAE
    rmse_train, mae_train = calculate_rmse_mae(train_df['Actual_Train'], train_df.iloc[:, 1])
    rmse_test, mae_test = calculate_rmse_mae(test_df['Actual_Test'], test_df.iloc[:, 1])

     # 打印 RMSE 和 MAE 到控制台
    print(f'{model_name} - Train RMSE: {rmse_train:.4f}, Train MAE: {mae_train:.4f}')
    print(f'{model_name} - Test RMSE: {rmse_test:.4f}, Test MAE: {mae_test:.4f}')

    # 设置图形大小，确保适合期刊发布
    plt.figure(figsize=(8, 8), dpi=1200)

    # 创建 JointGrid 对象
    g = sns.JointGrid(height=8)

    # 绘制中心的散点图
    sns.scatterplot(x=train_df['Actual_Train'], y=train_df.iloc[:, 1], color=palette['Train'], ax=g.ax_joint, alpha=0.5, s=50, label='Train')
    sns.scatterplot(x=test_df['Actual_Test'], y=test_df.iloc[:, 1], color=palette['Test'], ax=g.ax_joint, alpha=0.5, s=50, label='Test')

    # 添加边缘的柱状图
    sns.histplot(train_df['Actual_Train'], kde=False, color=palette['Train'], alpha=0.5, ax=g.ax_marg_x)
    sns.histplot(test_df['Actual_Test'], kde=False, color=palette['Test'], alpha=0.5, ax=g.ax_marg_x)


    # 在右上角添加命中率HR文本
    ax = g.ax_joint
    ax.text(0.95, 0.1, f'Train HR = {hr_train:.3f}', transform=ax.transAxes, fontsize=10,        
            verticalalignment='bottom', horizontalalignment='right', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white", linewidth=0.5))
    ax.text(0.95, 0.05, f'Test HR = {hr_test:.3f}', transform=ax.transAxes, fontsize=10,        
            verticalalignment='bottom', horizontalalignment='right', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white", linewidth=0.5))

    # 在左上角添加模型名称文本
    ax.text(0.75, 0.99, f'Model = {model_name}', transform=ax.transAxes, fontsize=10,        
            verticalalignment='top', horizontalalignment='left', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white", linewidth=0.5))

    # 添加中心线
    min_val = min(train_df['Actual_Train'].min(), test_df['Actual_Test'].min())
    max_val = max(train_df['Actual_Train'].max(), test_df['Actual_Test'].max())
    ax.plot([min_val, max_val], [min_val, max_val], c="black", alpha=0.7, linestyle='--', label='x=y')

     # 绘制 HR 上限线和下限线
    ax.plot([min_val, max_val], [min_val + 0.002, max_val + 0.002], color='orange', linestyle='--', label='HR Upper Limit (0.002)')
    ax.plot([min_val, max_val], [min_val - 0.002, max_val - 0.002], color='orange', linestyle='--', label='HR Lower Limit (0.002)')


    # 确保坐标轴线正确显示
    g.ax_joint.spines['right'].set_color('none')  # 右侧不显示
    g.ax_joint.spines['top'].set_color('none')    # 顶部不显示
    g.ax_joint.spines['left'].set_color('black')  # 左侧显示
    g.ax_joint.spines['bottom'].set_color('black')# 底部显示

    # 保持坐标轴刻度
    g.ax_joint.tick_params(axis="x", which="both", bottom=True, top=False)
    g.ax_joint.tick_params(axis="y", which="both", left=True, right=False)
    
    # 设置坐标轴标签
    g.ax_joint.set_xlabel('Actual Value', fontsize=12)  # 横坐标名称
    g.ax_joint.set_ylabel('Predicted Value', fontsize=12)  # 纵坐标名称

    # 显示图例
    ax.legend(fontsize=10, frameon=False)

    # 显示绘图
    plt.tight_layout()  # 自动调整图形以避免元素重叠
    plt.show()

In [ ]:
model=rf_best_model
explainer = shap.TreeExplainer(model)  # 对于树模型，使用TreeExplainer
shap_values_train = explainer.shap_values(X_train_scaled)  # 计算训练集的SHAP值
# 绘制训练集的SHAP汇总图
plt.figure(figsize=(8, 6), dpi=120)
shap.summary_plot(shap_values_train, X_train_scaled, feature_names=X_train_scaled.columns, 
                  plot_type="dot", max_display=18,show=False,cmap='coolwarm')
plt.tight_layout()  # 自动调整布局
plt.show()

In [ ]:
# 计算SHAP绝对值
shap_values_abs = np.abs(shap_values_train)  # 取每个SHAP值的绝对值

# 绘制绝对重要性图
shap.summary_plot(shap_values_abs, X_train_scaled, feature_names=X_train_scaled.columns, plot_type='bar',max_display=30)

In [ ]:
# 计算特征的重要性
feature_importances = np.abs(shap_values_train).mean(axis=0)  # 取 SHAP 值的平均绝对值
feature_importances = feature_importances / feature_importances.sum() * 100  # 转化为百分比
feature_names = X_train_scaled.columns.tolist()

# 合并低于 2% 的特征为一个“其他”类别
threshold = 2.0  # 设置百分比阈值
other_importance = feature_importances[feature_importances < threshold].sum()  # 计算低于阈值的总和
feature_names = [feature_names[i] for i in range(len(feature_importances)) if feature_importances[i] >= threshold]
feature_importances = [feature_importances[i] for i in range(len(feature_importances)) if feature_importances[i] >= threshold]

# 将 "Other" 添加到最后
feature_names.append("Other")
feature_importances.append(other_importance)

# 对特征重要性排序（从大到小，其他固定在最后）
sorted_indices = np.argsort(feature_importances[:-1])[::-1]  # 排序，排除 "Other"
feature_importances = [feature_importances[i] for i in sorted_indices] + [feature_importances[-1]]
feature_names = [feature_names[i] for i in sorted_indices] + ["Other"]

# 自定义独特颜色列表，从红色为首开始
unique_colors = [
    "#FF6F61", "#FFA07A", "#FFD700","#FFB6C1","#FFDAB9", "#87E200", "#32CD32",'#2DC16C',
    "#CA3AA4",'#98469C','#358CD3','#19A2D9','#6490B4',"#D3D3D3"   # 确保"Other"为浅灰色
]
colors = unique_colors[:len(feature_importances)]

# 调整最大的扇形高度
explode = [0.1 if i < 2 else 0 for i in range(len(feature_importances))]  # 前三个突出，其他为 0

# 绘制环形饼图
fig, ax = plt.subplots(figsize=(10, 8), dpi=1200)

wedges, texts, autotexts = ax.pie(
    feature_importances,
    labels=None,  # 不显示标签
    autopct='%1.1f%%',  # 百分比格式
    startangle=90,
    counterclock=False,  # 顺时针排列
    colors=colors,
    pctdistance=1.1,  # 百分比放在环外
    explode=explode  # 仅调整最大扇形的高度
)

# 设置数字颜色和扇形颜色一致
for autotext, wedge in zip(autotexts, wedges):
    autotext.set_fontsize(10)
    autotext.set_fontweight("bold")
    autotext.set_color(wedge.get_facecolor())  # 设置数字颜色为扇形颜色

# 添加中心白色圆形，形成环形效果
centre_circle = plt.Circle((0, 0), 0.50, fc='white')  # 内圈更小
fig.gca().add_artist(centre_circle)

# 图例
plt.legend(
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1),
    labels=[
        f"{name}" for name in feature_names
    ],
    title="Features",
    fontsize=10,
    title_fontsize=12
)

# 标题
plt.tight_layout()
plt.show()

In [ ]:
# 计算特征的重要性
feature_importances = np.abs(shap_values_train).mean(axis=0)  # 取 SHAP 值的平均绝对值
feature_importances = feature_importances / feature_importances.sum() * 100  # 转化为百分比
feature_names = X_train_scaled.columns.tolist()

# 对特征按重要性排序（从大到小）
sorted_indices = np.argsort(feature_importances)[::-1]  # 从大到小排序
feature_importances = feature_importances[sorted_indices]
feature_names = [feature_names[i] for i in sorted_indices]
# 自定义独特颜色列表，从红色为首开始
unique_colors = ["#FF6F61", "#FFA07A", "#FFD700","#FFB6C1","#FFDAB9", "#87E200", "#32CD32","#20B2AA"]  # 确保"Other"为浅灰色]
colors = unique_colors[:len(feature_importances)]
# 调整最大的扇形高度
explode = [0.1 if i < 5 else 0 for i in range(len(feature_importances))]  # 前三个突出，其他为 0

# 绘制环形饼图
fig, ax = plt.subplots(figsize=(10, 8), dpi=1200)

wedges, texts, autotexts = ax.pie(
    feature_importances,
    labels=None,  # 不显示标签
    autopct='%1.1f%%',  # 百分比格式
    startangle=90,
    counterclock=False,  # 顺时针排列
    colors=colors,
    pctdistance=1.1,  # 百分比放在环外
    explode=explode  # 仅调整最大扇形的高度
)

# 设置数字颜色和扇形颜色一致
for autotext, wedge in zip(autotexts, wedges):
    autotext.set_fontsize(10)
    autotext.set_fontweight("bold")
    autotext.set_color(wedge.get_facecolor())  # 设置数字颜色为扇形颜色

# 添加中心白色圆形，形成环形效果
centre_circle = plt.Circle((0, 0), 0.50, fc='white')  # 内圈更小
fig.gca().add_artist(centre_circle)

# 图例
plt.legend(
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1),
    labels=[
        f"{name}" for name in feature_names
    ],
    title="Features",
    fontsize=10,
    title_fontsize=12
)

# 标题
plt.tight_layout()
plt.show()

In [ ]:
# 选择两个特征进行相互作用分析，这里选择特征'Feature_1' 和 'Feature_2'
feature1_index = features_train.columns.get_loc('Lime')
feature2_index = features_train.columns.get_loc('Slag')

# 获取这两个特征的原始值和对应的SHAP值
feature1_values = features_train['Lime'].values
feature2_values = features_train['Slag'].values
shap_values_feature1 = shap_values_train[:,feature1_index]
shap_values_feature2 = shap_values_train[:,feature2_index]

# 计算这两个特征的SHAP值的和
shap_values_sum = shap_values_feature1 + shap_values_feature2

# 绘制3D散点图
fig = plt.figure(figsize=(10, 8), dpi=1200)
ax = fig.add_subplot(111, projection='3d')

# 绘制散点，颜色由SHAP值的和决定，红色表示正值，蓝色表示负值
sc = ax.scatter(feature1_values, feature2_values, shap_values_sum, c=shap_values_sum, cmap='coolwarm', s=50)

# 为大于0的点绘制虚线
for i in range(len(shap_values_sum)):
    if shap_values_sum[i] > 0:
        # 虚线的XY底面投影（shap值为0时的投影位置）
        ax.plot([feature1_values[i], feature1_values[i]], 
                [feature2_values[i], feature2_values[i]], 
                [shap_values_sum[i], np.min(shap_values_sum)], color='grey', linestyle='--', alpha=0.5)

# 设置坐标轴标签
ax.set_xlabel('Lime Value',fontsize=15)
ax.set_ylabel('Slag Value',fontsize=15)

# 调整视角，设定方向（例如：俯视的角度）
ax.view_init(elev=20, azim=30)

# 添加颜色条
cbar = fig.colorbar(sc)
cbar.set_label('SHAP Value Sum')

# 展示图形
plt.tight_layout()
plt.show()